<a href="https://colab.research.google.com/github/murdo25/DeepLearningZoo/blob/master/t5_for_sentence_classification_from_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nlp
!pip install tqd
!pip install transformers
!pip install sentencepiece


ERROR: Could not find a version that satisfies the requirement tqd (from versions: none)
ERROR: No matching distribution found for tqd


In [39]:
from nlp import Dataset
import csv


def read(data_path):
    open_file = open(data_path, 'r').readlines()

    lines = []
    # Strips the newline character 
    for line in open_file: 
        line = line.strip()
        lines.append(line)
    return lines


def clean_csv(data_path):
    with open(data_path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        inputs = []
        targets = []

        for row in reader:
            inputs.append('Is this sentence about finance? ' + row[0])
            if(row[1] == '0.0'):
                targets.append(" Answer: NO")
            else:
                targets.append(" Answer: YES")
    return inputs, targets 

def build_dataset(data_file):
    inputs, targets  = clean_csv(data_file)

    datapoints = {}
    datapoints['input_text']= inputs
    datapoints['target_text']= targets

    assert len(datapoints['target_text']) == len(datapoints['input_text']), "incorrect data distribution"
    return Dataset.from_dict(datapoints)


In [3]:
import tqdm
import torch
import nlp
from transformers import T5Tokenizer
import json
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional
import numpy as np

tokenizer = T5Tokenizer.from_pretrained('t5-small')


In [40]:
# tokenize the examples
def convert_to_features(example_batch):
    # print("batch", example_batch)
    input_encodings  = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=128)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=12)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }
    # print('encodings', encodings)
    return encodings


train_dataset = build_dataset('train.csv')
valid_dataset = build_dataset('test.csv')

print("train",train_dataset)
print("valid",valid_dataset)


# map convert_to_features batch wise
train_dataset = train_dataset.map(convert_to_features, batched=True)
valid_dataset = valid_dataset.map(convert_to_features, batched=True)

# print("train dataset", train_dataset)
# set the tensor type and the columns which the dataset should return
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

# cach the dataset, so we can load it directly for training

torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')


from transformers import T5ForConditionalGeneration, T5Tokenizer, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)


logger = logging.getLogger(__name__)

from data_classes import T2TDataCollator, ModelArguments, DataTrainingArguments


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


train Dataset(features: {'input_text': Value(dtype='string', id=None), 'target_text': Value(dtype='string', id=None)}, num_rows: 6718)
valid Dataset(features: {'input_text': Value(dtype='string', id=None), 'target_text': Value(dtype='string', id=None)}, num_rows: 1216)


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [47]:
def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.
    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

    # we will load the arguments from a json file, 
    #make sure you save the arguments in at ./args.json
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    tokenizer = T5Tokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    model = T5ForConditionalGeneration.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    train_dataset  = torch.load(data_args.train_file_path)
    valid_dataset = torch.load(data_args.valid_file_path)

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=T2TDataCollator(),
    )
    
    # Training
    if training_args.do_train:
        loss = trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        print("loss: ", loss)
        trainer.save_model()
        tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results

args_dict = {
  "num_cores": 8,
#   'training_script': 'train_t5_squad.py',
#   "model_name_or_path": './pretrained/gpu/checkpoint-24000',
  "model_name_or_path": 't5-small',
  "tokenizer_name": 't5-small',
  "max_len": 128 ,
  "target_max_len": 12,
  "output_dir": './models/gpu',
  "overwrite_output_dir": True,
  "per_gpu_train_batch_size": 32,
  "per_gpu_eval_batch_size": 32,
  "gradient_accumulation_steps": 4,
  "learning_rate": 1e-4,
  "tpu_num_cores": 8,
  "do_train": True,
  "num_train_epochs": 64,
  "save_steps": 200
}

with open('args.json', 'w') as f:
  json.dump(args_dict, f)

"""Start training!"""
main()

01/20/2021 20:51:10 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
01/20/2021 20:51:10 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=./models/gpu, overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, gradient_accumulation_steps=4, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=64, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Jan20_20-51-10_5211ff82f0a5, logging_first_step=False, logging_steps=500, save_steps=200, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O1, fp16_backend=auto, local_rank=-1, tpu_num_cores=8, tpu_metrics_debug=False, debug=Fals

Step,Training Loss
500,0.195700
1000,0.049200
1500,0.030900
2000,0.019000
2500,0.012700
3000,0.009200


loss:  TrainOutput(global_step=3328, training_loss=0.0484048376003137, metrics={'train_runtime': 2060.2451, 'train_samples_per_second': 1.615, 'total_flos': 19976595910164480, 'epoch': 63.99})


{}

In [48]:
# F1: https://en.wikipedia.org/wiki/F-score

## SQuAD evaluation script. Modifed slightly for this notebook
from data_classes import ModelArguments
from transformers import HfArgumentParser
parser = HfArgumentParser(ModelArguments)

from collections import Counter
import string
import re
import argparse
import json
import sys
import os
import torch
import nlp
from transformers import T5ForConditionalGeneration, T5Tokenizer, set_seed 
from tqdm.auto import tqdm
from os import listdir
set_seed(42)


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truth, prediction in zip(gold_answers, predictions):
      total += 1
      exact_match += exact_match_score(prediction, ground_truth)
      f1 += f1_score(prediction, ground_truth)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

def clean(result):
    result = result.replace("<pad>","")
    result = result.replace("</s>", "")
    result = result.strip()
    result = result.lower()
    return result

checkpoints = "models/gpu/"

for checkpoint in sorted(listdir(checkpoints)):
    # print("checkpoint:", checkpoint)
    if(checkpoint.split("-")[0] != "checkpoint"):
        continue
    # if(checkpoint.split("-")[1] != "83000"):
    #     continue

    print(checkpoint.split("-")[1])

    print("checkpoint:", checkpoint)

    model = T5ForConditionalGeneration.from_pretrained(checkpoints + checkpoint).to('cuda')

    tokenizer = T5Tokenizer.from_pretrained('t5-small')

    valid_dataset = torch.load('valid_data.pt')
    dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=256)

    answers = []
    for batch in dataloader:
      outs = model.generate(input_ids=batch['input_ids'].to('cuda'), 
                            attention_mask=batch['attention_mask'].to('cuda'),
                            max_length=16,
                            early_stopping=True)
      outs = [tokenizer.decode(ids) for ids in outs]
      answers.extend(outs)

    predictions = []
    references = []
    for ref, pred in zip(valid_dataset, answers):
      predictions.append(clean(pred))
      references.append(clean(tokenizer.decode(ref['target_ids'])))

    print(checkpoint, evaluate(references, predictions))

1000
checkpoint: checkpoint-1000
checkpoint-1000 {'exact_match': 88.15789473684211, 'f1': 94.07894736842105}
1200
checkpoint: checkpoint-1200
checkpoint-1200 {'exact_match': 87.66447368421052, 'f1': 93.83223684210526}
1400
checkpoint: checkpoint-1400
checkpoint-1400 {'exact_match': 88.32236842105263, 'f1': 94.16118421052632}
1600
checkpoint: checkpoint-1600
checkpoint-1600 {'exact_match': 88.56907894736842, 'f1': 94.2845394736842}
1800
checkpoint: checkpoint-1800
checkpoint-1800 {'exact_match': 88.07565789473684, 'f1': 94.03782894736842}
200
checkpoint: checkpoint-200
checkpoint-200 {'exact_match': 83.38815789473684, 'f1': 91.69407894736842}
2000
checkpoint: checkpoint-2000
checkpoint-2000 {'exact_match': 88.32236842105263, 'f1': 94.16118421052632}
2200
checkpoint: checkpoint-2200
checkpoint-2200 {'exact_match': 87.5, 'f1': 93.75}
2400
checkpoint: checkpoint-2400
checkpoint-2400 {'exact_match': 87.58223684210526, 'f1': 93.79111842105263}
2600
checkpoint: checkpoint-2600
checkpoint-2600

In [ ]:
# !pip install torchsummary
# print(batch['input_ids'].to('cuda').shape)
# from torchsummary import summary
# summary(model, input_size=[17,128])
# summary(model, input_size=[32128, 512])

# print(model)

In [ ]:
!zip -r checkpoint-24000.zip models/gpu/checkpoint-24000

In [ ]:
!rm checkpoint-83000.zip

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
# !cp checkpoint-83000.zip gdrive/MyDrive/colab/
!cp checkpoint-24000.zip gdrive/MyDrive/colab/

In [ ]:
!cp gdrive/MyDrive/colab/checkpoint-24000.zip .

In [ ]:
!unzip checkpoint-20000.zip 

In [ ]:
!ls -lt gdrive/MyDrive/colab